In [20]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import SimpleQueue
from functools import cmp_to_key
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache

In [2]:
sys.setrecursionlimit(1500)

In [3]:
with open("19-input", "r") as file:
    lines = file.readlines()
data_raw = [line.replace("\n", "") for line in lines]
data_raw = "\n".join(data_raw)
data_raw

'bcj{s>2236:A,s<1879:A,x<3217:R,R}\ndx{s>2913:fjr,m>3556:A,a>845:A,qmf}\nllc{m<375:A,A}\nsk{s>1852:A,s>1779:A,A}\nkkb{a<3232:R,x>2370:A,R}\nqpz{s>2002:R,s<1970:A,A}\nxh{m<1123:hxp,hbs}\nhvs{x>833:A,a>3047:A,m<2545:A,A}\nbn{m<884:R,m<1819:A,x<191:R,A}\nmg{m>3212:tbn,a>2969:bvv,m<2894:hn,dv}\nzxl{m>1434:zm,kpk}\nndg{m<2376:A,m>2763:A,A}\nrzf{x<493:A,A}\nbvj{s>2051:kpv,a<430:jsq,rz}\nknm{m<972:R,R}\njds{s<2286:R,x<544:A,a>2810:A,A}\nrdg{m>1477:bzn,m<1233:srv,tlm}\njg{x>938:cz,a<1498:rnn,nvj}\nzx{s>418:rgl,x>3567:R,jxd}\nzg{m<471:A,a<2572:rzf,s>2871:A,R}\njcs{s<2971:R,s>3598:A,x<3099:R,R}\nrk{s<3628:jg,bgx}\ndj{s<2872:R,x>871:llc,A}\ntsk{m>2811:R,a<2339:R,R}\njv{m>760:fkk,s<2822:rzk,mr}\nhhd{m>1117:A,A}\nghz{a>1392:dvd,hhd}\ndn{x>968:A,s<1630:R,R}\nnts{a>80:A,a>64:A,R}\nxn{m<573:R,a<3414:R,R}\nqc{m>1686:dvp,bm}\nmlh{m>692:A,x<2902:A,A}\nxfk{m>2842:R,R}\nvrh{a>2782:R,R}\ndcf{s<3052:A,m<2489:R,s<3448:A,A}\ntv{x>2354:tp,s>2429:R,ppm}\ngt{m>2262:A,s>3008:R,A}\nbs{a>2819:mhl,pgx}\ndg{x<1711:A,a

In [21]:
test_data_raw = r"""px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}"""


Step = namedtuple("Step", ["direction", "number", "color"])
def preprocess_data (data):
    # dtype='U10'
    processes, parts = data.split("\n\n")[0], data.split("\n\n")[1]

    def eval_action_string (action):
        if len(action.split(":")) == 1:
            return {"target": action}
        else:
            rule = action.split(":")[0]
            target = action.split(":")[1]
            if len(rule.split("<")) > 1:
                op = "<"
            elif len(rule.split(">")) > 1:
                op = ">"
            else: 
                raise Exception("Unknown operator")
            
            rule = {"var": rule.split(op)[0], "op": op, "val": int(rule.split(op)[1]), "target": target  }
            return rule

    # print (eval_action_string('a<2006:qkq'))
    # print (eval_action_string('m>2090:A'))
    # print (eval_action_string('rfg'))
    processes = [{process.split("{")[0]: process.split("{")[1].replace("}", "")    } for process in processes.split("\n")]
    processes = [ {id: [eval_action_string(action)  for action in actions.split(",")] for id, actions in process.items()} for process in processes]
    processes = dict(ChainMap(*processes))

    parts = [[part_component.replace("{", "").replace("}", "") for part_component in part.split(",")] for part in parts.split("\n") ]
    parts = [{part_component.split("=")[0]: int(part_component.split("=")[1])  for part_component in part} for part in parts ]
    # rows = [row.split(" ") for row in data.split("\n")]
    # steps = [Step(direction, int(num), color.split("(")[1].split(")")[0]) for direction, num, color in rows]
    return processes, parts

test_data = preprocess_data(test_data_raw)
test_data

({'hdj': [{'var': 'm', 'op': '>', 'val': 838, 'target': 'A'},
   {'target': 'pv'}],
  'gd': [{'var': 'a', 'op': '>', 'val': 3333, 'target': 'R'}, {'target': 'R'}],
  'qqz': [{'var': 's', 'op': '>', 'val': 2770, 'target': 'qs'},
   {'var': 'm', 'op': '<', 'val': 1801, 'target': 'hdj'},
   {'target': 'R'}],
  'in': [{'var': 's', 'op': '<', 'val': 1351, 'target': 'px'},
   {'target': 'qqz'}],
  'crn': [{'var': 'x', 'op': '>', 'val': 2662, 'target': 'A'},
   {'target': 'R'}],
  'qkq': [{'var': 'x', 'op': '<', 'val': 1416, 'target': 'A'},
   {'target': 'crn'}],
  'qs': [{'var': 's', 'op': '>', 'val': 3448, 'target': 'A'},
   {'target': 'lnx'}],
  'rfg': [{'var': 's', 'op': '<', 'val': 537, 'target': 'gd'},
   {'var': 'x', 'op': '>', 'val': 2440, 'target': 'R'},
   {'target': 'A'}],
  'lnx': [{'var': 'm', 'op': '>', 'val': 1548, 'target': 'A'},
   {'target': 'A'}],
  'pv': [{'var': 'a', 'op': '>', 'val': 1716, 'target': 'R'}, {'target': 'A'}],
  'px': [{'var': 'a', 'op': '<', 'val': 2006, 't

In [22]:
data = preprocess_data(data_raw)
data

({'pbx': [{'var': 's', 'op': '>', 'val': 3731, 'target': 'R'},
   {'target': 'R'}],
  'df': [{'var': 'm', 'op': '>', 'val': 2377, 'target': 'A'},
   {'var': 'x', 'op': '<', 'val': 2393, 'target': 'R'},
   {'var': 's', 'op': '<', 'val': 794, 'target': 'A'},
   {'target': 'A'}],
  'lkl': [{'var': 'a', 'op': '>', 'val': 3513, 'target': 'R'},
   {'target': 'A'}],
  'xvb': [{'var': 's', 'op': '<', 'val': 257, 'target': 'A'},
   {'var': 'm', 'op': '<', 'val': 824, 'target': 'A'},
   {'target': 'R'}],
  'spr': [{'var': 'm', 'op': '>', 'val': 1479, 'target': 'R'},
   {'var': 'a', 'op': '>', 'val': 2918, 'target': 'R'},
   {'var': 'm', 'op': '<', 'val': 1424, 'target': 'A'},
   {'target': 'A'}],
  'tk': [{'var': 'a', 'op': '>', 'val': 2726, 'target': 'grr'},
   {'target': 'nsj'}],
  'nth': [{'var': 'a', 'op': '>', 'val': 63, 'target': 'R'}, {'target': 'A'}],
  'pr': [{'var': 'x', 'op': '<', 'val': 863, 'target': 'R'},
   {'var': 'x', 'op': '<', 'val': 882, 'target': 'A'},
   {'var': 's', 'op': 

In [27]:
def solution (data, verbose=False):
    processes, parts = data
    results = []
    for part in parts:
        process_id = 'in'
        while not process_id in ("A", "R"):
            process_actions = processes[process_id]
            for action in process_actions:
                if not "var" in action:
                    process_id = action["target"]
                    break
                else:
                    if action["op"] == "<":
                        if part[action["var"]] < action["val"]:
                            process_id = action["target"]
                            break
                    elif action["op"] == ">":
                        if part[action["var"]] > action["val"]:
                            process_id = action["target"]
                            break
        if process_id == "A":
            results.append(sum(part.values()))
    return results



sol = solution(data)
# sol = solution(test_data, verbose=True)

print(sol)
display(sum(sol))


[7751, 3917, 3869, 3679, 4357, 4587, 3460, 7550, 5017, 3502, 5431, 3398, 4494, 3593, 9230, 3408, 3033, 1010, 3648, 2969, 2529, 4529, 3731, 5558, 5373, 3853, 6757, 3201, 5289, 4394, 3069, 3681, 5845, 3911, 2686, 1648, 5576, 5758, 1920, 2831, 3452, 4339, 2878, 6899, 4940, 6174, 3733, 3800, 5327, 5942, 4294, 3067, 4784, 7190, 4930, 2330, 5493, 3002, 2946, 5680, 4361, 7402, 5014, 1906, 6188, 1892, 3728, 2424, 4713, 4990, 3247, 6020, 2654, 2408, 3118, 1999, 2854, 4681, 3909, 2769, 4621, 2261, 1749, 4947, 3279, 5908, 4426, 4038, 7278, 5546, 1919, 3092, 2670, 4462, 2823, 4966, 2672, 2362, 4669, 5014, 4518]


420739

# Part 2

In [78]:
Node = namedtuple("Node", ["id", "split", "children"])

def solution (data, verbose=False):
    processes, parts = data
    results = []

    # print(processes["in"])
    
    def invert_action (action, tar):
        inverted_dict = dict(action)
        if inverted_dict["op"] == ">":
            inverted_dict["op"] = "<="
        elif inverted_dict["op"] == "<":
            inverted_dict["op"] = ">="
        inverted_dict["target"] = tar
        return inverted_dict


    def next_node (process_id, split):
        
        if process_id in ("A", "R"):
            return Node(process_id, split, [])
        else:
            children = []
            prev_inverted_actions = []
            for action in processes[process_id]:
                next_action = []
                if "var" in action:
                    next_action = [action]
                
                applied_actions = prev_inverted_actions + next_action
                if next_action:
                    prev_inverted_actions += [invert_action(next_action[0], action["target"])]
                children.append(next_node(action["target"], applied_actions))

            return Node(process_id, split, children)

    root = next_node("in", "")
    accepted_paths = []

    def find_accepted_paths (node, steps):
        if node.id == "A":
            accepted_paths.append(steps + [node.split])
        elif node.id != "R":
            for child in node.children:
                find_accepted_paths(child, steps + [node.split])

    find_accepted_paths(root, [])
    if verbose:
        print(len(accepted_paths))
    
    accepted_paths =  [[unpacked_part for path_part in path for unpacked_part in path_part if path_part != "" ] for path in accepted_paths ]

    combinations_paths = []

    for path in accepted_paths:
        if verbose:
            print("\n", path)
        # split up conditions
        x_conds = {"<": [], ">":[], ">=":[], "<=":[]}
        m_conds = {"<": [], ">":[], ">=":[], "<=":[]}
        a_conds = {"<": [], ">":[], ">=":[], "<=":[]}
        s_conds = {"<": [], ">":[], ">=":[], "<=":[]}
        for split in path:
            if split["var"] == "x":
                x_conds[split["op"]].append(split)
            elif split["var"] == "m":
                m_conds[split["op"]].append(split)  
            elif split["var"] == "a":
                a_conds[split["op"]].append(split) 
            elif split["var"] == "s":
                s_conds[split["op"]].append(split) 
        
        # turn >=, <= into >, < resp
        for conds in [x_conds, m_conds, a_conds, s_conds]:
            for cond in conds["<="]:
                new_cond = dict(cond)
                new_cond["op"] = "<"
                new_cond["val"] += 1
                conds["<"].append(new_cond)
            
            for cond in conds[">="]:
                new_cond = dict(cond)
                new_cond["op"] = ">"
                new_cond["val"] -= 1
                conds[">"].append(new_cond)

        restraints = {}
        for var, conds in [("x", x_conds), ("m", m_conds), ("a", a_conds), ("s", s_conds)]:
            higher_restraints = [cond["val"] for cond in conds["<"]]
            if higher_restraints:
                upper_bound = min(higher_restraints)
            else: 
                upper_bound = 4001
            lower_restraints = [cond["val"] for cond in conds[">"]]
            if lower_restraints:
                lower_bound = max(lower_restraints)
            else: 
                lower_bound = 0
            if verbose:
                print(var, "between", lower_bound, "and", upper_bound, "(exclusive);", (upper_bound -1) - lower_bound, "combinations" )

            restraints[var] = {"<": upper_bound, ">": lower_bound}

        combinations = 1
        for restraint in restraints.values():
            combinations *= (restraint["<"]-1) - restraint[">"]

        combinations_paths.append(combinations)
        if verbose:
            print(combinations)
    
    return combinations_paths


sol = solution(data)
# sol = solution(test_data, verbose=True)

print(sol)
display(sum(sol))


[1670845512000, 1211018872000, 525532603200, 1379437293240, 192435147576, 44072531776, 114863995296, 31357787580, 29776722660, 630145234208, 148222306512, 143243262624, 28468510784, 226990959360, 1477101491640, 170329536000, 175834320480, 33048763020, 330042570144, 111493467522, 34106924016, 33390780690, 17640412440, 63890196480, 24937786368, 11891793606, 8290512384, 32127617984, 169248485208, 177749946792, 123043768320, 1551742720, 174151451264, 138459384000, 84806180928, 35654250240, 49321712832, 38695328000, 58042992000, 256769850144, 146234003520, 93479693648, 193981213440, 200043126360, 155848000000, 65481000000, 756514500000, 249129416820, 69035566950, 265006833150, 117829479600, 241615885500, 166041117000, 66171831120, 62020751820, 429191413060, 172044140400, 184469550540, 82759051368, 67874647610, 81335502094, 417124254324, 762227054664, 119804950125, 104895889665, 33258441600, 19893486600, 56816504400, 93332007900, 48011219084, 26726216775, 18911029596, 1440481680000, 95159092

130251901420382